In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from glob import glob
from datetime import datetime
import sys
import os
sys.path.append('/Users/w76m851/github_workspace/spicy-snow/')
from datetime import datetime

from spicy_snow.processing.snow_index import calc_delta_cross_ratio, calc_delta_gamma, \
    clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth
from spicy_snow.processing.wet_snow import id_newly_wet_snow, id_wet_negative_si, \
    id_newly_frozen_snow, flag_wet_snow


In [13]:
def calc_rmse(lidar, insar):
    return(np.sqrt(((lidar-insar)**2).sum()/len(lidar)))

dirs = sorted(glob('/Users/w76m851/OneDrive - Montana State University/spicy_s1_stacks/param_regional/*'))
site_names = [s.split('regional/')[1] for s in dirs]
results_df = pd.DataFrame(index=site_names, columns=['min_rmse','params'])

for name in site_names[5:]:
    print(datetime.now(), f'-- starting {name}')
    files = sorted(glob(f'/Users/w76m851/OneDrive - Montana State University/spicy_s1_stacks/param_regional/{name}/*.npy'))
    if files:
        lidar = np.load(files[-1])
        min_rmse = 9999
        min_params = ''
        for f in files[:-1]:
            insar = np.load(f)
            rmse = calc_rmse(lidar, insar)
            if rmse < min_rmse:
                min_rmse = rmse
                min_params = f.split('/')[-1]

        results_df.loc[name] = [min_rmse, min_params]


# npy_dirs = glob('/Users/w76m851/OneDrive - Montana State University/spicy_s1_stacks/param_regional/*/')
# npy_dirs
# for d in npy_dirs[:1]:
#     files = sorted(glob(d+'*.npy', recursive=True))

# print(d, files)


2023-05-02 12:04:36.127700 -- starting Frasier_2021-03-19
2023-05-02 12:38:09.949334 -- starting Little_Cottonwood_2021-03-18
2023-05-02 12:38:09.960612 -- starting Mores_2020-02-09
2023-05-02 13:12:26.217013 -- starting Mores_2021-03-15


In [4]:
results_df.to_csv('rmse_results_20230502_part2.csv')

In [12]:
np.load(files[-1])

array([], dtype=float32)